In [1]:
import os
ROOT = os.path.join('..', '..')
import sys
sys.path.append(ROOT)
# 
import numpy as np

from src.models import SVDCollaborativeFiltering

In [2]:
from src.utils import read_ml, ml_train_test_split

ml_complete_df, ml_users_df, ml_ratings_df, ml_movies_df, ml_genres_df = read_ml()

MAX_K = 15
ml_ratings_train_df, ml_ratings_test_df = ml_train_test_split(ml_ratings_df=ml_ratings_df, min_user_test_samples=MAX_K)

In [3]:
recommender = SVDCollaborativeFiltering(ml_movies_df, ml_users_df)
recommender.fit(ml_ratings_train_df)

c:\Users\UADulinaTe\Desktop\UCU_5K2S_Recommender_Systems_Project\experiments\7_factorization_svd\..\..\src\models\factorization_svd.py:27: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  utility_matrix_df = utility_matrix_df.fillna(global_mean)


In [4]:
metrics = recommender.evaluate(ml_ratings_test_df, k=MAX_K)

100%|██████████| 442/442 [00:06<00:00, 63.53it/s]


In [5]:
metrics.keys()

dict_keys(['precision@k', 'recall@k', 'f1@k', 'average_precision', 'reciprocal_rank', 'NDCG'])

In [6]:
print("Predictive metrics")
print("Average Precision@K:", round(np.mean(metrics["precision@k"]), 4))
print("Average Recall@K:", round(np.mean(metrics["recall@k"]), 4))
print("Average F1@K:", round(np.mean(metrics["f1@k"]), 4))
print("Average precision:", round(np.mean(metrics["average_precision"]), 4))

print()
print("Best Precision@K:", round(np.max(metrics["precision@k"]), 4))
print("Best Recall@K:", round(np.max(metrics["recall@k"]), 4))
print("Best F1@K:", round(np.max(metrics["f1@k"]), 4))
print("Best average precision", round(np.max(metrics["average_precision"]), 4))
print()
print("Ranking metrics")
print("MRR:", round(np.mean(metrics["reciprocal_rank"]), 4))
print("Average NDCG:", round(np.mean(metrics["NDCG"]), 4))

Predictive metrics
Average Precision@K: 0.3106
Average Recall@K: 0.0597
Average F1@K: 0.0895
Average precision: 0.3513

Best Precision@K: 1.0
Best Recall@K: 0.28
Best F1@K: 0.35
Best average precision 1.0

Ranking metrics
MRR: 0.6609
Average NDCG: 0.6115


In [6]:
example_user = recommender.predicted_ratings_df.copy().iloc[2]
example_user = example_user.reset_index()
example_user.columns = ["MovieID", "Score"]
example_user.sort_values(by="Score", ascending=False, inplace=True)

In [7]:
example_user

,MovieID,Score
14,2857,4.012112
164,295,3.841767
287,317,3.837986
327,49,3.837287
59,592,3.756970
...,...,...
744,1543,3.069926
787,1087,3.066118
1231,1561,3.065362
1459,152,3.049017


In [8]:
example_user = example_user.iloc[:15]

In [9]:
ml_user_ratings_test_df = ml_ratings_test_df[ml_ratings_test_df["UserID"] == 3086]
example_user["is_relevant"] = example_user["MovieID"].isin(
    ml_user_ratings_test_df["MovieID"])
ranked_list = list(example_user['is_relevant'].apply(lambda x: 1 if x == True else 0))
ideal_ranked_list = ranked_list.copy()
ideal_ranked_list = sorted(ideal_ranked_list, reverse=True)
print(ranked_list)
print(ideal_ranked_list)
DCG = 0
IDCG = 0
for i in range(len(ranked_list)):
    DCG += (2**ranked_list[i] - 1) / float(np.log2(i + 2))
    IDCG += (2**ideal_ranked_list[i] - 1) / float(np.log2(i + 2))
print(DCG)
print(IDCG)

[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1.142810832044357
2.1309297535714578


C:\Users\UADulinaTe\AppData\Local\Temp\ipykernel_20232\4106382330.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  example_user["is_relevant"] = example_user["MovieID"].isin(


In [10]:
DCG/IDCG

0.5362968113467822

In [37]:
type(float(np.log2(i + 2)))

float

In [12]:
np.log2(i + 2)

4.0

In [11]:
from src.metrics import NDCG

NDCG(example_user, ml_ratings_test_df=ml_ratings_test_df, user_id=3086)

c:\Users\UADulinaTe\Desktop\UCU_5K2S_Recommender_Systems_Project\experiments\7_factorization_svd\..\..\src\metrics.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rec_df["is_relevant"] = rec_df["MovieID"].isin(


0.5362968113467822